# Amazon Comprehend의 데이터셋 준비와 Custom entity recognizer 실행하기

### [(원본)](https://github.com/aws-samples/amazon-comprehend-custom-entity/blob/master/1-AWS-Comprehend-Custom-Entities.ipynb)

이 노트북은 Amazon Comprehend의 custom entities 의 학습 데이터셋을 위한 준비하는 방법에 대해 살펴보도록 하겠습니다.

Custom entity recongizer 모델을 생성하는 방법에 대한 추가적인 정보는 다음 링크를 참조하시기 바랍니다. 
https://docs.aws.amazon.com/comprehend/latest/dg/training-recognizers.html

In [38]:
# library imports
import re
import numpy as np
import pandas as pd
import matplotlib
import csv



이번 예제는 twitter dataset을 사용할 예정으로 다음 링크에서 데이터셋을 다운로드하여 압축을 푼 후, ./data 폴더에 업로드를 합니다. https://www.kaggle.com/thoughtvector/customer-support-on-twitter

업로드 한 데이데셋을 읽어 확인합니다

In [41]:
tweets = pd.read_csv('./data/twcs.csv',encoding='utf-8')
print(tweets.shape)
tweets.head()

(2811774, 7)


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


<a id='data-wrangling'></a>

## 데이터 전처리

tweet 데이터 셋은 흥미롭게도 약 3백만 개의 트윗을 포함하고 tweet 작성자와 tweet이 질의인지 대답인지 ("inbound" 컬럼)를 포함하고 있습니다.
만약 tweet이 질의이면 "response_tweet_id" 컬럼에는 support team의 답변이 제공됩니다. 

각 행에서 질의-응답 데이터의 결과값을 얻기위해서 [이 링크](https://www.kaggle.com/soaxelbrooke/first-inbound-and-response-tweets)에서 코드를 가져왔습니다.

In [42]:
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]

QnR = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')

# Filter to only outbound replies (from companies)
QnR = QnR[QnR.inbound_y ^ True]
print(f'Data shape: {QnR.shape}')
QnR.head()

Data shape: (794299, 14)


,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,17,NaN,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",19,NaN,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,NaN,20.0


In [50]:
#데이터프레임에서 필요한 컬럼만 추출합니다

QnR = QnR[["author_id_x","created_at_x","text_x","author_id_y","created_at_y","text_y"]]
QnR.head(5)

,author_id_x,created_at_x,text_x,author_id_y,created_at_y,text_y
0,115712,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...
1,115712,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...
2,115712,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...
3,115713,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,sprintcare,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...
4,115715,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",sprintcare,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...


## 통신사 tweet만 필터링하기

이 예제에서는 스마트폰 디바이스를 인식하기위한 custom entity를 생성합니다. 
우선 데이터프레임에서 T-Mobile과 Sprint의 tweet만 필터링하여 telco tweets 데이터를 만듭니다. 

In [47]:
tweet_telco = QnR[QnR["author_id_y"].isin(["TMobileHelp", "sprintcare"])]

필터링된 데이터에서 질문과 응답의 text 데이터를 결합하여 하나의 text 컬럼으로 생성합니다. 

In [ ]:
tweet_telco['text'] = tweet_telco['text_x']+ ' | ' + tweet_telco['text_y']

telco tweets 데이터를 csv 파일로 저장합니다. 

In [51]:
tweet_telco['text'].to_csv('./data/tweet_telco.csv', encoding='utf-8', index=False)

데이셋을 생성하기 위해서는 DEVICE라는 새 클래스의 Entity 목록을 제공해야 합니다. 

관련 Entity를 찾기 위해서는, 코퍼스를 word2vec 모델로 로드하고 유사한 키워드 목록을 생성할 수 있습니다. 
이 기술은 두 번째 예에서 사용될 것입니다.

우리는 Device를 찾기 위해 스마프톤의 다른 철자 목록을 생성할 것입니다. 

In [53]:
sphones = ['iPhone X', 'iPhoneX', 'iphoneX', 'Samsung Galaxy', 'Samsung Note', 'iphone', 'iPhone', 'android', 'Android']

df_entity_list = pd.DataFrame(sphones, columns=['Text'])

In [54]:
df_entity_list

,Text
0,iPhone X
1,iPhoneX
2,iphoneX
3,Samsung Galaxy
4,Samsung Note
5,iphone
6,iPhone
7,android
8,Android


DEVICE 클래스 레이블로 다른 컬럼을 추가합니다. 이것은 Amazon Comprehend의 학습 데이터셋의 요구 사양입니다. 

자세한 내용은 다음 링크를 확인해주시기 바랍니다. 

https://docs.aws.amazon.com/comprehend/latest/dg/cer-entity-list.html


In [55]:
df_entity_list['Type'] = 'DEVICE'


In [56]:
df_entity_list.head()

,Text,Type
0,iPhone X,DEVICE
1,iPhoneX,DEVICE
2,iphoneX,DEVICE
3,Samsung Galaxy,DEVICE
4,Samsung Note,DEVICE


학습 데이터파일을 생성합니다. 

In [ ]:
# 파일이 크면 학습이 늦어지므로 일부만 학습하도록 합니다
#tweet_telco['text'].tail(5000).to_csv('./data/raw_txt.csv', encoding='utf-8', index=False)

tweet_telco['text'].to_csv('./data/raw_txt.csv', encoding='utf-8', index=False)

In [60]:
!head ./data/raw_txt.csv

"I feel like #Sprint hates me. 

Maybe I deserve consistent slow LTE. Thanks Sprint, you only give me a low amount of first world problems. 

I miss #TMobile &lt;/3 they sucked too, but not as bad as Sprint. | @803494 This sounds concerning. How long have you been experiencing slow speeds on your device? Does this happen in specific locations? Do you notice this indoors or outdoors? - JF"
.@115990 you all are pathetic I have been with you all 10 years and have been with uber 3 with a discount now you are forcing me back to #Sprint | @176582 Hey there! We'd love to see you as part of our family. Please check our great offers in this link: https://t.co/dwtDgGbWzU . - EG
"Very #disappointed again by the services of @116297 @115817 and @115911 - We keep paying and yet no one delivers.... #complaint #technews #tmobile #asurion #ups - Thanks for nothing! | @221395 Hi Athan, DM us and tell us what's going on. We'll make sure you get what you're expecting. *JoanO"
"@TMobileHelp @115911 I'm in 


Entity 목록 파일을 생성합니다. 

In [63]:
df_entity_list.to_csv('./data/entity_list.csv', encoding='utf-8', index=False)


In [61]:
!head ./data/entity_list.csv

Text,Type
iPhone X,DEVICE
iPhoneX,DEVICE
iphoneX,DEVICE
Samsung Galaxy,DEVICE
Samsung Note,DEVICE
iphone,DEVICE
iPhone,DEVICE
android,DEVICE
Android,DEVICE


원본 telco tweet 데이터셋에서 테스트 파일을 만듭니다.


In [ ]:
tweet_telco['text'].tail(100).to_csv('./data/telco_test.csv', encoding='utf-8', index=False)

## S3에 파일 업로드

S3에 생성된 entry_list.csv, raw_txt.csv, telco_test.csv 파일을 업로드합니다.  


![title](./img/s3.png)

## 모델 학습하기

Amazon Comprehend 콘솔을 통해 Custom entity recognizer의 Job을 생성하겠습니다. 
custom entity 설정은 다음과 같습니다. 

![title](./img/ter.png)

Amazon Comprehend가 생성한 S3버킷에 접근하기 위해서는 다음과 같이 IAM role을 생성합니다. 
![title](./img/iam.png)

## Custom entity 모델 테스트하기

(1) 생성된 Custom entity 모델을 테스트하기 위해서 Amazon Comprehend 콘솔의 Analysis jobs를 통해 분석 job을 생성합니다. 

![title](./img/ajob.png)

(2) Comprehend CLI를 통해서 S3에 올려놓은 테스트 파일에 대한 분석 job을 실행할 수 있다. 다음은 그 예제입니다. 

In [ ]:
!aws comprehend start-entities-detection-job \
     --entity-recognizer-arn "arn:aws:comprehend:us-east-1:{account number}:entity-recognizer/twitter-device-cer" \
     --job-name Test \
     --data-access-role-arn "arn:aws:iam::{account number}:role/service-role/AmazonComprehendServiceRole-cer-test" \
     --language-code en \
     --input-data-config "S3Uri=s3://{bucket name}/custom_entities/telco_test.csv" \
     --output-data-config "S3Uri=s3://{bucket name}/custom_entities/output/" \
     --region "us-east-1"


참고로 CLI로 모델을 테스트하기 위해서는 IAM role 설정이 추가적으로 필요합니다.
data-access-role-arn 의 role은 해당 버킷에 PutObject 권한이 필요하고, 노트북 인스턴스의 role은 iam:PassRole 설정이 필요합니다.
관련 내용은 다음 링크를 참고하시기 바랍니다. 
https://docs.aws.amazon.com/comprehend/latest/dg/access-control-managing-permissions.html

테스트가 완료되면 --output-data-config 경로에 압축된 output json 파일이 생성됩니다. 
![title](./img/output.png)

## Output 결과 확인하기
 
Ouptput 결과는 json 파일을 직접 열어 확인하거나 AWS Glue와 Athena를 통해 확인가능합니다. 
AWS Glue의 crawler를 통해 S3의 output 파일을 읽어 Database를 만들고 Athena를 통해 조회가 가능합니다. 
다음은 Glue의 Crawler 설정 예제입니다. 
![title](./img/glue.gif)

Athena에서는 다음과 같은 query를 통해 Custom entity 인식 결과를 Device별로 조회할 수 있습니다. 

"SELECT col3, count(col3) FROM "comprehend"."telco_device_test_json" group by col3;"

![title](./img/test.png)

위 결과 중, "ipone"은 데이터셋에 태깅된 Entity 목록에 포함되지 않으나 Comprehend는 어느 정도 정확한 신뢰도를 가지고 이를 선택할 수 있습니다. 


### 참고
https://aws.amazon.com/blogs/machine-learning/getting-started-with-amazon-comprehend-custom-entities/
https://aws.amazon.com/blogs/machine-learning/build-a-custom-entity-recognizer-using-amazon-comprehend/
 